As a data scientist at a telecommunications company, you have been tasked with developing a classification model to predict customer churn (cancellation of subscriptions) based on their historical behavior and demographic information. The company wants to understand which customers are likely to churn in order to develop targeted customer retention programs.

**Dataset:**
You have been provided with a dataset named "Telco-Customer-Churn.csv". Each row in the dataset represents a customer, and each column contains attributes related to the customer's services, account information, and demographic details. The dataset includes the following information:

1.   Customers who left within the last month (Churn)
2.   Services that each customer has signed up for (phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies)
3.   Customer account information (how long they've been a customer, contract type, payment method, paperless billing, monthly charges, and total charges)
4.   Demographic information about customers (gender, age range, and whether they have partners and dependents)

**Objective:**
Your task is to develop a classification model that can accurately predict whether a customer is likely to churn based on the available information. The model should be trained on a portion of the dataset and evaluated on the remaining unseen data.

**Approach:**
1.   Load and preprocess the dataset: Load the dataset and handle any missing values. Encode categorical variables using appropriate encoding techniques.
2.   Split the dataset: Split the dataset into training and testing sets to train the model on a subset of data and evaluate its performance on unseen data.
3.   Feature scaling: Perform feature scaling on the numerical features to ensure they are on a similar scale, using techniques like StandardScaler.
4.   Model selection and training: Train multiple classification models such as K-Nearest Neighbors, Random Forest, Support Vector Machines, and Logistic Regression.
5.   Model evaluation: Evaluate the trained models on the testing set using appropriate evaluation metrics such as accuracy.
6.   Select the best model: Compare the performance of different models and select the best-performing model based on the chosen evaluation metric.
7.   Predict churn: Use the selected model to predict churn for new, unseen data.

**Deliverables:**


1.   Preprocessed dataset without missing values.
2.   Trained classification models with their respective evaluation metrics.
3.   The best-performing model for churn prediction.
4.   Predictions of churn for new, unseen data.

**Note:** The final model can be used by the telecommunications company to identify customers who are likely to churn, enabling them to take proactive measures to retain those customers.

Remember to adapt the problem statement and approach as needed, based on any specific requirements or modifications provided by the interviewer or the organization you are applying to.

In [39]:
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [40]:
data = pd.read_csv("churn_data.csv")

In [41]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [42]:
le = LabelEncoder()
ohe = OneHotEncoder()

In [43]:
data.isnull().any() #check for any null values; from output no null values seen
data.dropna(inplace = True)

In [44]:
categorical_columns = ["gender", "SeniorCitizen", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup",
                       "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "Churn"]

In [45]:
data_ohe_encoded_df = pd.get_dummies(data, columns = categorical_columns)

In [46]:
data_ohe_encoded_df.head()

,customerID,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,7590-VHVEG,1,29.85,29.85,True,False,True,False,False,True,...,False,False,False,True,False,False,True,False,True,False
1,5575-GNVDE,34,56.95,1889.5,False,True,True,False,True,False,...,True,False,True,False,False,False,False,True,True,False
2,3668-QPYBK,2,53.85,108.15,False,True,True,False,True,False,...,False,False,False,True,False,False,False,True,False,True
3,7795-CFOCW,45,42.30,1840.75,False,True,True,False,True,False,...,True,False,True,False,True,False,False,False,True,False
4,9237-HQITU,2,70.70,151.65,True,False,True,False,True,False,...,False,False,False,True,False,False,True,False,False,True


In [47]:
X = data_ohe_encoded_df.drop(['customerID', 'Churn_No', 'Churn_Yes' ], axis=1)

In [48]:
Y = data_ohe_encoded_df[['Churn_Yes']]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

In [50]:
knn = KNeighborsClassifier(n_neighbors = 3)
rfc = RandomForestClassifier(n_estimators = 7, criterion = 'entropy', random_state = 7)
svc = SVC()
lr = LogisticRegression()

In [51]:
#KNN Model
knn.fit(X_train, y_train)

ValueError: could not convert string to float: ' '